## ENV VARS
We require two environment variables:
1. **USER** _(default: current user)_
2. **HOST** _(default: docker-swarm-dev-manager)_

In [ ]:
export USER=$(/usr/bin/whoami)
echo $USER

In [ ]:
export HOST=docker-swarm
echo $HOST

In [ ]:
# WARNING: removes $HOST if exists
yes | docker-machine rm $HOST
echo
docker-machine ls

## CREATE HOST

In [ ]:
# create host
docker-machine create -d virtualbox $HOST

# 30+ seconds...

In [ ]:
# check host is up and running
docker-machine status $HOST

In [ ]:
# get IP for new host
export IP=$(docker-machine ip $HOST)
echo $IP

## HOSTS FILE
Either copy the IP and Domains below to your local hosts file, or follow instructions to [configure Dnsmaq](https://gist.github.com/eloypnd/5efc3b590e7c738630fdcf0c10b68072) (configure a resolver for "*dev*").

In [ ]:
# copy to /etc/hosts
echo sudo nano /etc/hosts
PORTAINER=portainer.$HOST.local
REGISTRY=registry.$HOST.local
echo $IP $PORTAINER
echo $IP $REGISTRY

In [ ]:
cat /etc/hosts

## INIT SWARM

In [ ]:
# init docker swarm
docker-machine ssh $HOST -- docker swarm init --advertise-addr $IP

In [ ]:
# export docker vars
export DOCKER_TLS_VERIFY=1
export DOCKER_HOST=tcp://$IP:2376
export DOCKER_CERT_PATH="/Users/$USER/.docker/machine/machines/$HOST"

# subsequent "docker" commands will execute on the swarm manager

## INSECURE REGISTRIES
We're going to deploy a registry without authentication.

We need to configure the manager to allow insecure registries.

We specify this in [Docker's daemon.json file](https://docs.docker.com/registry/insecure/)

In [ ]:
# create config for insecure-registries
mkdir temp
echo '{ "insecure-registries" : [ "'$IP':5000" ] }' > temp/daemon.json

# copy config to manager node
docker-machine scp ./temp/daemon.json $HOST:/home/docker/daemon.json
docker-machine ssh $HOST -- sudo cp /home/docker/daemon.json /etc/docker/daemon.json

In [ ]:
# make sure it got there correctly
docker-machine ssh $HOST cat /etc/docker/daemon.json

In [ ]:
# restart docker to accept new config
docker-machine ssh $HOST -- sudo /etc/init.d/docker restart
docker-machine ssh $HOST -- sudo /etc/init.d/docker status

## DOCKER REGISTRY
Our first stack to deploy is our Docker Registry. This will be required for other stacks requiring a registry for tagged images.

In [ ]:
cat registry/stack.yml

In [ ]:
# create overlay network: registry-net
docker network create -d overlay registry-net

In [ ]:
# mkdir for registry
docker-machine ssh $HOST -- sudo mkdir /mnt/registry

In [ ]:
# check that dir was created
docker-machine ssh $HOST -- ls /mnt | grep registry

In [ ]:
# deploy stack
docker stack deploy -c registry/stack.yml registry
echo http://$IP:5000/v2/ 

In [ ]:
docker-machine ssh $HOST -- docker service ps registry_registry

In [ ]:
# confirm 200 OK
curl -v http://$IP:5000/v2/

## TRAEFIK
Now that we have a registry, we can build and deploy Traefik.

Traefik is a reverse proxy we'll use for dynamic routing in subsequent stacks.

In [ ]:
cat traefik/stack.yml

In [ ]:
# create overlay network: traefik-net
docker network create -d overlay traefik-net

In [ ]:
cat traefik/Dockerfile

In [ ]:
cat traefik/traefik.toml

In [ ]:
# build
docker-compose -f traefik/stack.yml build

In [ ]:
# push
docker-compose -f traefik/stack.yml push

In [ ]:
# confirm latest
docker pull ${IP}:5000/traefik

In [ ]:
# deploy stack
docker stack deploy -c traefik/stack.yml traefik

In [ ]:
docker-machine ssh $HOST -- docker service ps traefik_traefik

In [ ]:
echo http://$IP/traefik
curl -s http://$IP/traefik/health | jq

## DOCKER REGISTRY FRONTEND

In [ ]:
cat docker-registry-frontend/stack.yml

In [ ]:
# deploy stack
docker stack deploy -c docker-registry-frontend/stack.yml registry

In [ ]:
docker-machine ssh $HOST -- docker service ps registry_registry-web

In [ ]:
echo http://$REGISTRY

## PORTAINER

In [ ]:
cat portainer/stack.yml

In [ ]:
# mkdir for portainer
docker-machine ssh $HOST -- sudo mkdir /mnt/portainer

In [ ]:
# check that dir was created
docker-machine ssh $HOST -- ls /mnt | grep portainer

In [ ]:
# deploy stack
docker stack deploy -c portainer/stack.yml portainer

In [ ]:
docker-machine ssh $HOST -- docker service ps portainer_portainer

In [ ]:
docker-machine ssh $HOST -- docker service ps portainer_agent

In [ ]:
# copy to /etc/hosts
echo http://$PORTAINER

## WORKER NODES

In [ ]:
JOIN_CMD=$(docker swarm join-token worker | grep -oEi '(docker swarm join .+)$')
echo $JOIN_CMD

In [ ]:
# worker nodes
W1=$HOST-worker1
W2=$HOST-worker2
echo $W1
echo $W2

In [ ]:
# WARNING: removes $HOST if exists
yes | docker-machine rm $W1
yes | docker-machine rm $W2

In [ ]:
# create worker 1
docker-machine create -d virtualbox $W1

In [ ]:
# check host is up and running
docker-machine status $W1

In [ ]:
# create worker 2
docker-machine create -d virtualbox $W2

In [ ]:
# check host is up and running
docker-machine status $W2

In [ ]:
docker-machine ip $W1
docker-machine ip $W2

In [ ]:
docker node ls

In [ ]:
# join worker 1
docker-machine ssh $W1 -- $JOIN_CMD

In [ ]:
# join worker 2
docker-machine ssh $W2 -- $JOIN_CMD

In [ ]:
# verify swarm nodes
docker node ls

## KEEPALIVED

In [ ]:
export VIP=192.168.99.99

In [ ]:
# create virtual IP on $HOST:
docker-machine ssh $HOST -- sudo ifconfig eth0:0 $VIP
docker-machine ssh $HOST -- ifconfig eth0:0
docker-machine ssh $HOST -- ping -c 1 $VIP

In [ ]:
# create virtual IP on $W1:
docker-machine ssh $W1 -- sudo ifconfig eth0:0 $VIP
docker-machine ssh $W1 -- ifconfig eth0:0
docker-machine ssh $W1 -- ping -c 1 $VIP

In [ ]:
# create virtual IP on $W2:
docker-machine ssh $W2 -- sudo ifconfig eth0:0 $VIP
docker-machine ssh $W2 -- ifconfig eth0:0
docker-machine ssh $W2 -- ping -c 1 $VIP

In [ ]:
# Enable IPVS module
mkdir temp
echo "modprobe ip_vs" > temp/rc.local

# copy config to $HOST
docker-machine scp ./temp/rc.local $HOST:/home/docker/rc.local
docker-machine ssh $HOST -- sudo cp /home/docker/rc.local /etc/rc.local

# copy config to $W1
docker-machine scp ./temp/rc.local $W1:/home/docker/rc.local
docker-machine ssh $W1 -- sudo cp /home/docker/rc.local /etc/rc.local

# copy config to $W2
docker-machine scp ./temp/rc.local $W2:/home/docker/rc.local
docker-machine ssh $W2 -- sudo cp /home/docker/rc.local /etc/rc.local

echo
echo verifying copies
echo $HOST: $(docker-machine ssh $HOST -- cat /etc/rc.local)
echo $W1: $(docker-machine ssh $W1 -- cat /etc/rc.local)
echo $W2: $(docker-machine ssh $W2 -- cat /etc/rc.local)

In [ ]:
docker-machine ssh $HOST -- modprobe ip_vs
docker-machine ssh $W1 -- modprobe ip_vs
docker-machine ssh $W2 -- modprobe ip_vs

In [ ]:
cat keepalived/stack.yml

In [ ]:
# default environment for keepalived

docker-machine ssh $HOST -- sudo mkdir /etc/keepalived
docker-machine ssh $HOST -- sudo mkdir /etc/keepalived/environment
docker-machine scp ./keepalived/environment/default.yaml $HOST:/home/docker/keepalived-env.yaml
docker-machine ssh $HOST -- sudo cp /home/docker/keepalived-env.yaml /etc/keepalived/environment/default.yaml
#docker-machine ssh $HOST -- sudo sed -i 's/KEEPALIVED_PRIORITY: 150/KEEPALIVED_PRIORITY: 100/' /etc/keepalived/environment/default.yaml && cat /etc/keepalived/environment/default.yaml

docker-machine ssh $W1 -- sudo mkdir /etc/keepalived
docker-machine ssh $W1 -- sudo mkdir /etc/keepalived/environment
docker-machine scp ./keepalived/environment/default.yaml $W1:/home/docker/keepalived-env.yaml
docker-machine ssh $W1 -- sudo cp /home/docker/keepalived-env.yaml /etc/keepalived/environment/default.yaml
#docker-machine ssh $W1 -- sudo sed -i 's/KEEPALIVED_PRIORITY: 150/KEEPALIVED_PRIORITY: 150/' /etc/keepalived/environment/default.yaml && cat /etc/keepalived/environment/default.yaml

docker-machine ssh $W2 -- sudo mkdir /etc/keepalived
docker-machine ssh $W2 -- sudo mkdir /etc/keepalived/environment
docker-machine scp ./keepalived/environment/default.yaml $W2:/home/docker/keepalived-env.yaml
docker-machine ssh $W2 -- sudo cp /home/docker/keepalived-env.yaml /etc/keepalived/environment/default.yaml
#docker-machine ssh $W2 -- sudo sed -i 's/KEEPALIVED_PRIORITY: 150/KEEPALIVED_PRIORITY: 200/' /etc/keepalived/environment/default.yaml && cat /etc/keepalived/environment/default.yaml


In [ ]:
docker-machine ssh $HOST -- sudo sed -i 's/\$\{KEEPALIVED_PRIORITY\}/100/' /etc/keepalived/environment/default.yaml
docker-machine ssh $W1 -- sudo sed -i 's/\$\{KEEPALIVED_PRIORITY\}/150/' /etc/keepalived/environment/default.yaml
docker-machine ssh $W2 -- sudo sed -i 's/\$\{KEEPALIVED_PRIORITY\}/200/' /etc/keepalived/environment/default.yaml


In [ ]:
echo $HOST: $(docker-machine ssh $HOST -- cat /etc/keepalived/environment/default.yaml | grep KEEPALIVED_PRIORITY)
echo $W1: $(docker-machine ssh $W1 -- cat /etc/keepalived/environment/default.yaml | grep KEEPALIVED_PRIORITY)
echo $W2: $(docker-machine ssh $W2 -- cat /etc/keepalived/environment/default.yaml | grep KEEPALIVED_PRIORITY)

In [ ]:
# deploy stack
docker stack deploy -c keepalived/stack.yml keepalived

In [ ]:
docker-machine ssh $HOST -- docker service ps keepalived_keepalived